# Setup

In [ ]:
%run setup.ipynb

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_merge = pd.read_csv('Datasets/df_merge.csv')

In [ ]:
df_merge = df_merge.rename(columns={"doc_speaker's-trustworthiness_index": "doc_speaker_trustworthiness_index"})

In [ ]:
df_merge["fre_label"]  = 0
for index, row in df_merge.iterrows():
    if row["fre"] >= 70:
        df_merge.loc[index, "fre_label"] = "alta"
    elif row["fre"] >= 40:
        df_merge.loc[index, "fre_label"] = "media"
    else:
        df_merge.loc[index, "fre_label"] = "bassa"


In [ ]:
df_merge["percentuale_parole_significative_label"]  = 0
for index, row in df_merge.iterrows():
    if row["percentuale_parole_significative"] >= 0.75:
        df_merge.loc[index, "percentuale_parole_significative_label"] = "alta"
    elif row["percentuale_parole_significative"] >= 0.5:
        df_merge.loc[index, "percentuale_parole_significative_label"] = "media"
    else:
        df_merge.loc[index, "percentuale_parole_significative_label"] = "bassa"


In [ ]:
df_merge["quartile_val"] = 0
for index,row in df_merge.iterrows():
    if row["quartile"] == "wrong_false":
        df_merge.loc[index,"quartile_val"] = 1
    elif row["quartile"] == "other_false":
        df_merge.loc[index,"quartile_val"] = 2
    elif row["quartile"] == "correct_false":
        df_merge.loc[index,"quartile_val"] = 3
    elif row["quartile"] == "other":
        df_merge.loc[index,"quartile_val"] = 4
    elif row["quartile"] == "wrong_true":
        df_merge.loc[index,"quartile_val"] = 5
    elif row["quartile"] == "other_true":
        df_merge.loc[index,"quartile_val"] = 6
    elif row["quartile"] == "correct_true":
        df_merge.loc[index,"quartile_val"] = 7
    else:
        print("Error")

In [ ]:
df_merge["doc_speaker_party_val"] = 0
for index,row in df_merge.iterrows():
    if row["doc_speaker_party"] == "DEM":
        df_merge.loc[index,"doc_speaker_party_val"] = 1

In [ ]:
# creo i vari df per le analisi
def crea_df_per_analisi(df_analisi):
    df_all = df_analisi.copy()

    df_false = df_analisi[df_analisi["sovra"] == "pants-on-fire"]
    df_true = df_analisi[df_analisi["sovra"] == "true"]

    df_wrong_false = df_analisi[df_analisi["quartile"] == "wrong_false"]
    df_correct_false = df_analisi[df_analisi["quartile"] == "correct_false"]
    df_wrong_true = df_analisi[df_analisi["quartile"] == "wrong_true"]
    df_correct_true = df_analisi[df_analisi["quartile"] == "correct_true"]

    df_wrong = df_wrong_false.append(df_wrong_true)
    df_correct = df_correct_true.append(df_correct_false)

    DF_no_quartili = [["df_all",df_all],["df_false",df_false],["df_true",df_true],["df_wrong",df_wrong],["df_correct",df_correct]]
    DF = [["df_all",df_all],["df_false",df_false],["df_true",df_true],["df_wrong",df_wrong],["df_correct",df_correct],["df_wrong_false",df_wrong_false],["df_correct_false",df_correct_false],["df_correct_true",df_correct_true],["df_wrong_true",df_wrong_true]]
    return df_all, df_false, df_true , df_wrong, df_correct, df_wrong_false, df_correct_false, df_correct_true, df_wrong_true, DF_no_quartili, DF

In [ ]:
df_all, df_false, df_true , df_wrong, df_correct, df_wrong_false, df_correct_false, df_correct_true, df_wrong_true, DF_no_quartili, DF = crea_df_per_analisi(df_merge)

# Analisi

In [45]:
variabili_interesse = ["scarto","quartile_val","doc_truthfulness_value"]
dimensioni_importanti = [# RQ2
                         "doc_speaker_party",
                         # RQ3
                         "doc_accuracy_index","doc_speaker_trustworthiness_index",
                         # RQ1
                         "age_ans_label","southern_border_ans_label","k_info_1_ans_label","k_ai_4_ans_label","s_com_1_v2_ans_label","s_info_4_ans_label"]
dimensioni_importanti1 = ["age_ans_label","southern_border_ans_label","taxes_ans_label","southern_border_ans_label","political_views_ans_label","consideration_ans_label","k_info_1_ans_label","k_ai_4_ans_label","s_com_1_v2_ans_label","s_info_4_ans_label"]
dimensioni_importanti2 = ["doc_speaker_party"]
dimensioni_importanti3 = ["doc_accuracy_index","doc_speaker_trustworthiness_index"]

In [ ]:
for c in dimensioni_importanti:
    print(c,df_all[c].isna().sum())

df_all = df_all.dropna(subset=dimensioni_importanti)

for c in dimensioni_importanti:
    print(c,df_all[c].isna().sum())

In [ ]:
for dim in variabili_interesse:
    print(dim,dim in df_all.columns)

In [ ]:
for dim in dimensioni_importanti:
    print(dim,dim in df_all.columns)

Ci sono alcune linee guida generali che possono essere utilizzate per interpretare i valori di ω²:

- Valori molto bassi: solitamente ω² inferiore a 0,01 indica che la variabile indipendente ha una relazione molto debole o trascurabile con la variabile dipendente.

- Valori bassi: ω² compreso tra 0,01 e 0,06 indica una relazione debole o piccola tra la variabile indipendente e la variabile dipendente.

- Valori medi: ω² compreso tra 0,06 e 0,14 indica una relazione moderata tra la variabile indipendente e la variabile dipendente.

- Valori alti: ω² compreso tra 0,14 e 0,39 indica una relazione moderatamente forte tra la variabile indipendente e la variabile dipendente.

- Valori molto alti: ω² superiore a 0,39 indica una relazione molto forte tra la variabile indipendente e la variabile dipendente.

In [ ]:
def crea_txt(dimensioni_importanti):
    txt = ""
    array = []
    for t in dimensioni_importanti:
        if "label" in t:
            txt = txt + f"""C({t}) + """
        else:   
            txt = txt + f"""{t} + """
        array.append([t])
    return txt[:-2]

In [ ]:
df_all

In [ ]:
df_all["doc_speaker_party_value"] = 0
for index,row in df_all.iterrows():
    if row["doc_speaker_party"] == "DEM":
        df_all.loc[index,"doc_speaker_party_value"] = 1
        



In [ ]:
index = "doc_speaker_party_value:doc_statement_date"
valori = index.split(":")
gr = []
for val in valori:
    if val[0:2] == "C(":
        val = val[2:-1]
        val = val[:-5]+"index"
    gr.append(val)
print(df_all[gr])

In [ ]:

# Creazione di due variabili di esempio
var1 = np.array([1, 2, 3, 4, 5])
var2 = np.array([6, 7, 8, 9, 10,5,2])

# Combina le due variabili in una singola variabile categorica
groups = np.concatenate([np.repeat('Group1', len(var1)), np.repeat('Group2', len(var2))])

# Crea un DataFrame per l'analisi
data = pd.DataFrame({'Values': np.concatenate([var1, var2]), 'Groups': groups})

# Applica la funzione pairwise_tukeyhsd
result = pairwise_tukeyhsd(data['Values'], data['Groups'])

# Stampa i risultati
print(result)

In [ ]:
data

In [ ]:

# Creazione di due variabili di esempio
var1 = np.array([1, 2, 3, 4, 5])
var2 = np.array([6, 7, 8, 9, 10,5,2])

# Combina le due variabili in una singola variabile categorica
groups = np.concatenate([np.repeat('Group1', len(var1)), np.repeat('Group2', len(var2))])

# Crea un DataFrame per l'analisi
data = pd.DataFrame({'Values': np.concatenate([var1, var2]), 'Groups': groups})

# Applica la funzione pairwise_tukeyhsd
result = pairwise_tukeyhsd(data['Values'], data['Groups'])

# Stampa i risultati
print(result)

In [ ]:
def create_df_for_turkey_test (df,varibile_interesse,variabili_indipendenti):
    n = len(variabili_indipendenti)
    
    if n == 1:
    
        a = []

        for val in df[variabili_indipendenti[0]].unique():
            gr = df[df[variabili_indipendenti[0]] == val][varibile_interesse]
            a.append([val,gr.values])
            
        groups = []
        for valore in a:
            groups = np.concatenate([groups,np.repeat(valore[0], len(valore[1]))])

        data = pd.DataFrame({'Values': np.concatenate([valore[1] for valore in a]), 'Groups': groups})
    
    
    elif n == 2:
    
        a = []

        for val1 in df_all[variabili_indipendenti[0]].unique():
            for val2 in df_all[variabili_indipendenti[1]].unique():
                gr = df_all[(df_all[variabili_indipendenti[0]] == val1) & (df_all[variabili_indipendenti[1]] == val2)][varibile_interesse]
                
                a.append([str(val1)+":"+str(val2),gr.values])
                
        groups = []
        for valore in a:
            groups = np.concatenate([groups,np.repeat(valore[0], len(valore[1]))])

        data = pd.DataFrame({'Values': np.concatenate([valore[1] for valore in a]), 'Groups': groups})
    
    elif n == 3:
        
        a = []
        
        for val1 in df_all[variabili_indipendenti[0]].unique():
            for val2 in df_all[variabili_indipendenti[1]].unique():
                for val3 in df_all[variabili_indipendenti[2]].unique():
                    gr = df_all[(df_all[variabili_indipendenti[0]] == val1) & (df_all[variabili_indipendenti[1]] == val2) & (df_all[variabili_indipendenti[2]] == val3)][varibile_interesse]
                    
                    a.append([str(val1)+":"+str(val2)+":"+str(val3),gr.values])
        
        groups = []
        for valore in a:
            groups = np.concatenate([groups,np.repeat(valore[0], len(valore[1]))])
        
        data = pd.DataFrame({'Values': np.concatenate([valore[1] for valore in a]), 'Groups': groups})
    
    
    return data

In [ ]:
create_df_for_turkey_test(df_all,"quartile_val",["doc_speaker_party","doc_statement_date"])

In [ ]:
create_df_for_turkey_test(df_all,"quartile_val",["doc_speaker_party"])

In [ ]:
df = df_all[["doc_speaker_party","doc_statement_date","quartile_val"]]
#df["gruppo"] = 0 

a = []

for val1 in df_all["doc_speaker_party"].unique():
    for val2 in df_all["doc_statement_date"].unique():
        gr = df_all[(df_all["doc_speaker_party"] == val1) & (df_all["doc_statement_date"] == val2)]["quartile_val"]
        
        a.append([str(val1)+":"+str(val2),gr.values])
        
groups = []
for valore in a:
    groups = np.concatenate([groups,np.repeat(valore[0], len(valore[1]))])

data = pd.DataFrame({'Values': np.concatenate([valore[1] for valore in a]), 'Groups': groups})

result = pairwise_tukeyhsd(data['Values'], data['Groups'])

print(result)
        

In [ ]:
data

In [ ]:
a[0]

In [ ]:
tukey_results = pairwise_tukeyhsd(endog=df_all["quartile_val"], 
                                    groups= df_all["doc_speaker_party_value"] + df_all["doc_statement_date"], 
                                    alpha=0.05)

In [ ]:
index = "doc_speaker_party_value:doc_statement_date"
valori = index.split(":")
gr = []
for val in valori:
    if val[0:2] == "C(":
        val = val[2:-1]
        val = val[:-5]+"index"
    gr.append(val)
print(gr)

In [ ]:
def calcola_anova(variabili,df_riferimento,txt,valore_soglia):

    for variabile_interesse in variabili:
        for df in df_riferimento:
            
            
            results = ols(
                f'''
                    {variabile_interesse} ~ {txt} """
                '''
                , data=df[1]).fit()
            
            aov_table = sm.stats.anova_lm(results, typ=2)
            anova_table_v = anova_table(aov_table)
            if anova_table_v["omega_sq"].max() > valore_soglia:
                print("-" * 100)
                print(f"""{variabile_interesse} ~ {txt} in {df[0]}""")
                display(anova_table_v.sort_values(by="omega_sq", ascending=False).head(3))
                
            for index, row in anova_table_v.iterrows():
                if row["omega_sq"] > valore_soglia:
                    print(index)
                    
                    if ":" in index:
                        valori = index.split(":")
                        gr = []
                        for val in valori:
                            if val[0:2] == "C(":
                                val = val[2:-1]
                                val = val[:-5]+"index"
                            gr.append(val)
                        print(gr)
                        
                        data = create_df_for_turkey_test(df[1],variabile_interesse,gr)
                        
                        tukey_results = pairwise_tukeyhsd(data['Values'], data['Groups'],alpha=0.05)
                        
                    else:
                    
                        if index[0:2] == "C(":
                            index = index[2:-1]
                            index = index[:-5]+"index"
                            
                        data = create_df_for_turkey_test(df[1],variabile_interesse,[index])
                        
                        tukey_results = pairwise_tukeyhsd(data['Values'], data['Groups'],alpha=0.05)
                    
                    df_turkey = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
                    display(df_turkey[df_turkey["reject"] == True].sort_values(by="p-adj", ascending=True).sort_values(by="meandiff", ascending=False).head(3))
        print("-" * 100)
        print("-" * 100)

In [ ]:
txt = "doc_speaker_party*doc_statement_date"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

Un'altra domanda: se voglio fare il turkey test userò la funzione pairwise_tukeyhsd. Come argomenti:
- endog = colonna variabile d'interesse 
- groups = colonne con variabili che devo controllare se 

In [ ]:
txt = "doc_speaker_party*doc_statement_date"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = crea_txt(dimensioni_importanti)
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [52]:
variabili_interesse

['scarto', 'quartile_val', 'doc_truthfulness_value']

In [55]:
txt = crea_txt(dimensioni_importanti1)
#txt = "age_ans_label*southern_border_ans_label*taxes_ans_label*southern_border_ans_label*political_views_ans_label*consideration_ans_label"
valore_soglia = 0.05
calcola_anova(['quartile_val', 'doc_truthfulness_value'],DF_no_quartili,txt,valore_soglia)

----------------------------------------------------------------------------------------------------
quartile_val ~ C(age_ans_label) + C(southern_border_ans_label) + C(taxes_ans_label) + C(southern_border_ans_label) + C(political_views_ans_label) + C(consideration_ans_label) + C(k_info_1_ans_label) + C(k_ai_4_ans_label) + C(s_com_1_v2_ans_label) + C(s_info_4_ans_label)  in df_correct


,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(k_info_1_ans_label),30.845526,2.0,15.422763,3.728999,0.028580,0.071623,0.051918
C(s_info_4_ans_label),11.543143,2.0,5.771571,1.395482,0.254071,0.026803,0.007524
C(southern_border_ans_label),7.682686,2.0,3.841343,0.928781,0.399530,0.017839,-0.001355


C(k_info_1_ans_label)


,group1,group2,meandiff,p-adj,lower,upper,reject
0,0.0,1.0,-1.5829,0.0194,-2.955,-0.2109,True


----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------


In [ ]:
txt = crea_txt(dimensioni_importanti2)
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = crea_txt(dimensioni_importanti3)
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt

In [ ]:
dimensioni_importanti

In [ ]:
txt = "age_ans_label*doc_speaker_trustworthiness_index"
valore_soglia = 0.06
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = "doc_speaker_party*doc_speaker_trustworthiness_index"
valore_soglia = 0.06
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = "doc_speaker_party*age_ans_label"
valore_soglia = 0.06
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = "doc_speaker_party*southern_border_ans_label*age_ans_label"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = "k_info_1_ans_label*k_ai_4_ans_label"
valore_soglia = 0.06
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
txt = "s_com_1_v2_ans_label*s_info_4_ans_label"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
doc_speaker_party*doc_speaker_trustworthiness_index

In [ ]:
txt = "(consideration_ans_index + political_views_ans_label) * doc_speaker_party"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
## interessante

txt = "(consideration_ans_index + political_views_ans_label) * (doc_speaker_party + doc_speaker_trustworthiness_index)"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
## interessante

txt = "(s_com_1_v2_ans_label + k_ai_4_ans_label) * (k_info_1_ans_label + s_info_4_ans_label)"
valore_soglia = 0.01
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
## interessante

txt = "(doc_accuracy_index) * (doc_speaker_party + doc_speaker_trustworthiness_index + age_ans_label + southern_border_ans_label)"
valore_soglia = 0.06
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
## interessante taxes_ans_label

txt = "(taxes_ans_label) * (doc_speaker_party + doc_speaker_trustworthiness_index + age_ans_label + southern_border_ans_label)"
valore_soglia = 0.1
calcola_anova(variabili_interesse,DF_no_quartili,txt,valore_soglia)

In [ ]:
df_all.columns

## Caratteristiche worker

##### doc_truthfulness_value

In [ ]:
## visione politica

txt = "doc_speaker_trustworthiness_index * (southern_border_ans_label + doc_speaker_party + political_views_ans_label + consideration_ans_label)"
valore_soglia = 0.1
calcola_anova(["doc_truthfulness_value"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

In [ ]:
df_false.groupby(["doc_speaker_trustworthiness_index"]).mean()["doc_truthfulness_value"]

In [ ]:
df_true.groupby(["doc_speaker_trustworthiness_index"]).mean()["doc_truthfulness_value"]

Se mi fido dell'autore tendo a dare in media dei valori più alti

In [ ]:
## domande

txt = "k_info_1_ans_label*(age_ans_label + southern_border_ans_label++k_ai_4_ans_label+s_com_1_v2_ans_label+s_info_4_ans_label)"
valore_soglia = 0.05
calcola_anova(["doc_truthfulness_value"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)


In [ ]:
## domande

txt = "k_info_1_ans_label*k_ai_4_ans_label "
valore_soglia = 0.05
calcola_anova(["doc_truthfulness_value"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)


In [ ]:
df_true.groupby(["k_info_1_ans_label"]).mean()["doc_truthfulness_value"]

In [ ]:
df_false.groupby(["k_info_1_ans_label"]).mean()["doc_truthfulness_value"]

La domanda è importante

##### scarto

In [ ]:
## visione politica

txt = "doc_speaker_party *  (southern_border_ans_label + doc_speaker_trustworthiness_index) * (political_views_ans_label + consideration_ans_label)"
valore_soglia = 0.05
calcola_anova(["scarto"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

In [ ]:
df_false.groupby(["quartile","doc_speaker_trustworthiness_index"]).mean()["scarto"]

In [ ]:
df_true.groupby(["doc_speaker_trustworthiness_index"]).mean()["scarto"]

Per le news vere:
- se mi fido poco dello scrittore allora tendo ad allontanarmi molto dal GT

Per le fake news:
- se mi fido poco dello scrittore allora tendo ad allontanarmi molto dal GT

In [ ]:
## domande

txt = "k_info_1_ans_label*k_ai_4_ans_label "
valore_soglia = 0.05
calcola_anova(["scarto"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)


##### quartile_val

In [ ]:
## visione politica

txt = "doc_speaker_party +  (southern_border_ans_label + doc_speaker_trustworthiness_index) * (political_views_ans_label + consideration_ans_label)"
valore_soglia = 0.05
calcola_anova(["quartile_val"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

SI nota che :
- doc_speaker_party è molto rilevante per l'individuazione dei gruppi
- doc_speaker_trustworthiness_index è molto importante per lo scarto e per doc_truthfulness_value => 

## Valutazione del Worker su Statement

In [ ]:
df_all.columns

In [ ]:
## interessante taxes_ans_label

txt = "doc_accuracy_index * doc_informativeness_index * doc_comprehensibility_index * doc_precision_index * doc_confidence_index * doc_unbiasedness_index" 
valore_soglia = 0.1
calcola_anova(variabili_interesse,[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

In [ ]:
## interessante taxes_ans_label

txt = "doc_accuracy_index + (doc_informativeness_index + doc_comprehensibility_index + doc_precision_index + doc_confidence_index) * doc_unbiasedness_index" 
valore_soglia = 0.1
calcola_anova(variabili_interesse,[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

In [ ]:
df = df_all.groupby(["quartile"]).mean()["doc_speaker_trustworthiness_index"]
df = df.reset_index(inplace=False)
df = df[(df["quartile"] != "other_true") & (df["quartile"] != "other_false") & (df["quartile"] != "other")]
df.sort_values(by="doc_speaker_trustworthiness_index", ascending=False)
display(df)
sns.barplot(x="quartile", y="doc_speaker_trustworthiness_index", data=df,order=["correct_false","wrong_false","wrong_true","correct_true"])

In [ ]:
df = df_all.groupby(["quartile"]).mean()["doc_accuracy_index"]
df = df.reset_index(inplace=False)
df = df[(df["quartile"] != "other_true") & (df["quartile"] != "other_false") & (df["quartile"] != "other")]
df.sort_values(by="doc_accuracy_index", ascending=False)
display(df)
sns.barplot(x="quartile", y="doc_accuracy_index", data=df,order=["correct_false","wrong_false","wrong_true","correct_true"])

- Con i statement FALSE ha più importanza la doc_speaker_trustworthiness_index
- Con i statement TRUE ha più importanza la doc_accuracy_index

## Caratteristiche Statement

##### doc_truthfulness_value

In [ ]:
## interessante sentiment_label e subjectivity_label e percentuale_parole_significative_label:subjectivity_label

txt = "sentiment_label * percentuale_parole_significative_label * subjectivity_label"
valore_soglia = 0.05
calcola_anova(["doc_truthfulness_value"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

In [ ]:
df_all.groupby(["sentiment_label","fre_label","percentuale_parole_significative_label"]).mean()[["doc_truthfulness_value"]].reset_index().sort_values(by="doc_truthfulness_value",ascending=False)

In [ ]:
df_all.groupby(["percentuale_parole_significative_label"]).mean()[["doc_truthfulness_value"]].reset_index()

In [ ]:
df_all.groupby(["sentiment_label"]).mean()[["doc_truthfulness_value"]].reset_index()

In [ ]:
df_all.groupby(["subjectivity_label"]).mean()[["doc_truthfulness_value"]].reset_index()

In [ ]:
sns.boxplot(x = "doc_truthfulness_value", y = "subjectivity", data = df_all)

##### scarto

In [ ]:
## interessante subjectivity_label:sentiment_label per lo scarto nel wrong 

# txt = "subjectivity_label * (sentiment_label + fre_label) * percentuale_parole_significative_label"
# valore_soglia = 0.05
# calcola_anova(["scarto"],[["df_all",df_all],["df_false",df_false],["df_true",df_true],["df_wrong",df_wrong],["df_correct",df_correct]],txt,valore_soglia)

In [ ]:
df_false.groupby(["scarto"]).mean()[["subjectivity","percentuale_parole_significative"]].reset_index()

##### quartile_val

In [ ]:
## TUTTO

## interessante 

txt = "(subjectivity_label  + sentiment_label) * percentuale_parole_significative_label * fre_label"
valore_soglia = 0.05
calcola_anova(["quartile_val"],[["df_all",df_all],["df_false",df_false],["df_true",df_true],["df_wrong",df_wrong],["df_correct",df_correct]],txt,valore_soglia)

In [ ]:
## SOLO FALSE

## interessante 

txt = "subjectivity_label  * (sentiment_label + percentuale_parole_significative_label + fre_label)"
valore_soglia = 0.1
calcola_anova(["quartile_val"],[["df_all",df_all],["df_false",df_false]],txt,valore_soglia)

In [ ]:
df_false.groupby("quartile").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity"]]

In [ ]:
## SOLO TRUE

## interessante 

txt = "subjectivity_label * fre_label * percentuale_parole_significative_label"
valore_soglia = 0.1
calcola_anova(["quartile_val"],[["df_false",df_true]],txt,valore_soglia)

In [ ]:
df_true.groupby("quartile").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity"]]

Notiamo che per quanto riguarda le fake news quelle che vengono azzeccate come tali:
- hanno una percentuale di parole significative più alta, un fre maggiore, sentiment negativo e poca subjectivity.
- contrario per le news che vengono classificate male (ricevono punteggio alto nonostante sono false)

Notiamo che per quanto riguarda le news vere quelle che vengono azzeccate come tali (correct):
- hanno un fre maggiore, sentiment minore e più subjectivity.
- contrario per le news positive che vengono classificate male (ricevono punteggio basso nonostante sono vere)


In [ ]:
def crea_box_plot(dimensioneX,dimensioneY):
    # Creazione dei boxplot utilizzando seaborn
    plt.figure(figsize=(15, 3))  # Dimensioni dell'immagine
    plt.subplot(1, 3, 1)  # Primo boxplot
    sns.boxplot(x = dimensioneX, y = dimensioneY,hue = "sovra", data = df_all[df_all["sovra"] != "other"])
    plt.title('ALL')

    plt.subplot(1, 3, 2)  # Secondo boxplot
    sns.boxplot(x = dimensioneX, y = dimensioneY,hue = "quartile", data = df_true[df_true["quartile"] != "other_true"])
    plt.title('TRUE')

    plt.subplot(1, 3, 3)  # Terzo boxplot
    sns.boxplot(x = dimensioneX, y = dimensioneY,hue = "quartile", data = df_false[df_false["quartile"] != "other_false"])
    plt.title('FALSE')

    plt.tight_layout()  # Per evitare sovrapposizioni di elementi
    plt.show()  # Mostra l'immagine con i tre boxplot

In [ ]:
quartili = ["wrong_false", "correct_false", "wrong_true", "correct_true"]
def crea_bar_plot_final(dimensioneX):
    # analisi suando prop test sulle proporzioni
    
    raccoglitore_dimensioni = crea_raccoglitore_dimensioni(df_all, [dimensioneX], quartili)
    tabella_p_value = costruisci_tabella_pvalue(raccoglitore_dimensioni, [dimensioneX], quartili,ignora_per_ptrend = ["doc_speaker_party"])
    significativi = filtra_siglificativi(tabella_p_value, [dimensioneX], quartili, 0.05,"pvalue_quartile_sovra").sort_values(by=["dimensione"], ascending=True)
    crea_barplot_all_anova(raccoglitore_dimensioni,tabella_p_value, dimensioneX,"valore")
    
    

In [ ]:
def crea_bar_plot(dimensioneX,dimensioneY):
    # Creazione dei boxplot utilizzando seaborn
    plt.figure(figsize=(15, 3))  # Dimensioni dell'immagine
    plt.subplot(1, 3, 1)  # Primo boxplot
    sns.barplot(x = dimensioneX, y = dimensioneY,hue = "sovra", data = df_all[df_all["sovra"] != "other"])
    plt.title('ALL')

    plt.subplot(1, 3, 2)  # Secondo boxplot
    sns.barplot(x = dimensioneX, y = dimensioneY,hue = "quartile", data = df_true[df_true["quartile"] != "other_true"])
    plt.title('TRUE')

    plt.subplot(1, 3, 3)  # Terzo boxplot
    sns.barplot(x = dimensioneX, y = dimensioneY,hue = "quartile", data = df_false[df_false["quartile"] != "other_false"])
    plt.title('FALSE')

    plt.tight_layout()  # Per evitare sovrapposizioni di elementi
    plt.show()  # Mostra l'immagine con i tre boxplot

In [ ]:
def crea_bar_plot(dimensioneX):
    # Creazione dei boxplot utilizzando seaborn
    plt.figure(figsize=(15, 3))  # Dimensioni dell'immagine
    plt.subplot(1, 3, 1)  # Primo boxplot
    data = df_all[dimensioneX].tolist()
    counts = pd.Series(data).value_counts().reset_index()
    sns.barplot(x='index', y=0, data=counts)
    plt.title('ALL')

    plt.subplot(1, 3, 2)  # Secondo boxplot
    data = df_true[dimensioneX].tolist()
    counts = pd.Series(data).value_counts().reset_index()
    sns.barplot(x='index', y=0, data=counts)
    plt.title('TRUE')

    plt.subplot(1, 3, 3)  # Terzo boxplot
    data = df_false[dimensioneX].tolist()
    counts = pd.Series(data).value_counts().reset_index()
    sns.barplot(x='index', y=0, data=counts)
    plt.title('FALSE')

    plt.tight_layout()  # Per evitare sovrapposizioni di elementi
    plt.show()  # Mostra l'immagine con i tre boxplot

In [ ]:

def crea_box_plot_statement(dimensioni_da_plottare):
    plt.figure(figsize=(15, 5))  # Dimensioni dell'immagine
    
    i = 1
    for dimensione in dimensioni_da_plottare:
    
        # Creazione dei boxplot utilizzando seaborn
        
        plt.subplot(1, len(dimensioni_da_plottare), i)  # Primo boxplot
        sns.boxplot(palette = "pastel",x="quartile", y=dimensione, data=df_all,order=["correct_false","wrong_false","wrong_true","correct_true"])
        sns.stripplot(palette = "dark",x="quartile", y=dimensione, data=df_all, jitter=0.2, size=5, alpha=0.95,order=["correct_false","wrong_false","wrong_true","correct_true"])
        plt.xticks(rotation=90)
        plt.title(dimensione)
        i += 1
        
    plt.tight_layout()  # Per evitare sovrapposizioni di elementi
    plt.show()  # Mostra l'immagine con i tre boxplot

### RIASSUMENDO

- se i worker si fidano dell'autore tendono a dare valutazioni più alte in media (sia se lo statement è true sia se è fake)

In [ ]:
## trust dello scrittore

txt = "(doc_precision_index + doc_completeness_index + doc_informativeness_index)+ doc_unbiasedness_index  + doc_speaker_trustworthiness_index"
valore_soglia = 0.1
calcola_anova(["doc_truthfulness_value"],[["df_all",df_all],["df_true",df_true],["df_false",df_all]],txt,valore_soglia)

In [ ]:
crea_box_plot("doc_speaker_trustworthiness_index","doc_truthfulness_value")

- se lo statement è TRUE: più mi fido dell'autore più mi avvicino al GT
- se lo statement è FAKE: se non mi fido dell'autore tendo ad azzeccare che è una FAKE

In [ ]:
crea_box_plot("doc_speaker_trustworthiness_index","scarto")

- gli statement scritti da DEMOCRATICI ricevono valutazioni in media più alte (indipendentemente dal tipo di statement) 

In [ ]:
for df in [["df_all",df_all],["df_false",df_false],["df_true",df_true]]:
    print("-" * 100)
    print(df[0])
    display(df[1].groupby(by="doc_speaker_party").mean()["doc_truthfulness_value"])

- il partito dello scrittore dello statement è rilevante per individuare il gruppo dello statement
=> ciò è anche confermato dal prop-test

In [ ]:
crea_bar_plot_final("doc_speaker_party")

- età e doc_speaker_party posso essere combinate per determinare quartile_val

In [ ]:
txt = "(consideration_ans_index + political_views_ans_label + doc_speaker_party) * age_ans_label"
valore_soglia = 0.05
calcola_anova(["quartile_val"],[["df_all",df_all],["df_false",df_false],["df_true",df_true]],txt,valore_soglia)

In [ ]:
for df in [["df_all",df_all],["df_false",df_false],["df_true",df_true]]:
    print("-" * 100)
    print(df[0])
    display(df[1].groupby(by="age_ans_label").mean()["doc_truthfulness_value"])

In [ ]:
df_true[df_true["quartile"] != "other_true"].groupby(by=["age_ans_label","quartile"]).mean()["doc_truthfulness_value"].unstack().plot(kind="bar",figsize=(15, 3))

- se lo statement viene giudicato come poco accurato si tenderà a dare una valutazione bassa in media
- se invece lo statement viene giudicato come molto accurato si tenderà a dare una valutazione alta in media

In [ ]:
## accuratezza del testo

txt = "(doc_precision_index + doc_completeness_index + doc_informativeness_index)+ doc_unbiasedness_index + doc_accuracy_index + doc_speaker_trustworthiness_index"
valore_soglia = 0.1
calcola_anova(["doc_truthfulness_value"],[["df_all",df_all],["df_true",df_true],["df_false",df_all]],txt,valore_soglia)

In [ ]:
crea_box_plot("doc_accuracy_index","doc_truthfulness_value")

In [ ]:
df = df_all.groupby(["quartile"]).mean()["doc_accuracy_index"]
df = df.reset_index(inplace=False)
df = df[(df["quartile"] != "other_true") & (df["quartile"] != "other_false") & (df["quartile"] != "other")]
df.sort_values(by="doc_accuracy_index", ascending=False)
display(df)
sns.barplot(x="quartile", y="doc_accuracy_index", data=df,order=["correct_false","wrong_false","wrong_true","correct_true"])

- Con i statement FALSE ha più importanza la doc_speaker_trustworthiness_index
- Con i statement TRUE ha più importanza la doc_accuracy_index

In [ ]:
## confronto tra TRUE e FALSE

txt = "doc_accuracy_index + doc_speaker_trustworthiness_index"
valore_soglia = 0.1
calcola_anova(["doc_truthfulness_value"],[["df_true",df_true],["df_false",df_false]],txt,valore_soglia)

- analizzando il testo dello statement:

In [ ]:
crea_box_plot_statement(["percentuale_parole_significative","fre","sentiment", "subjectivity"])

Notiamo che per quanto riguarda le news false quelle che vengono azzeccate come tali:
- hanno una percentuale di parole significative più alta, un fre maggiore, sentiment negativo e poca subjectivity.
- contrario per le news che vengono classificate male (ricevono punteggio alto nonostante sono false)

In [ ]:
## STATEMENT FALSE

txt = "(subjectivity_label  + sentiment_label) * percentuale_parole_significative_label * fre_label"
valore_soglia = 0.05
calcola_anova(["quartile_val"],[["df_all",df_all],["df_false",df_false]],txt,valore_soglia)

In [ ]:
df_false.groupby("quartile").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity"]].reset_index(inplace=False)

Notiamo che per quanto riguarda le news vere quelle che vengono azzeccate come tali (correct):
- hanno un fre maggiore, sentiment minore e più subjectivity.
- contrario per le news positive che vengono classificate male (ricevono punteggio basso nonostante sono vere)


In [ ]:
## STATEMENT VERI

txt = "(subjectivity_label  + sentiment_label) * percentuale_parole_significative_label * fre_label"
valore_soglia = 0.1
calcola_anova(["quartile_val"],[["df_all",df_all],["df_true",df_true]],txt,valore_soglia)

In [ ]:
df_true.groupby("quartile").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity"]].reset_index(inplace=False)

##### RELAZIONI

Verifico se lo scrittore di un determinato partito politico ha delle caratteristiche specifiche con cui scrive lo statement

In [ ]:
## relazione tra il partito dello scrittore e le caratteristiche del testo

txt = "(subjectivity_label * sentiment_label * fre_label  + numero_parole_frase)* percentuale_parole_significative_label "
valore_soglia = 0.12
calcola_anova(["doc_speaker_party_val"],[["df_all",df_all],["df_true",df_true],["df_false",df_all]],txt,valore_soglia)

In [ ]:
df_all.groupby("doc_speaker_party").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity","numero_parole_frase"]].T

In [ ]:
df_false.groupby("doc_speaker_party").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity","numero_parole_frase"]].T

In [ ]:
df_true.groupby("doc_speaker_party").mean()[["percentuale_parole_significative","fre","sentiment", "subjectivity","numero_parole_frase"]].T

In [ ]:
## relazione tra l'accuratezza di un testo e le altre caratteristiche del testo

txt = "(doc_comprehensibility_index + doc_confidence_index + doc_precision_index + doc_completeness_index + doc_informativeness_index) * doc_unbiasedness_index"
valore_soglia = 0.05
calcola_anova(["doc_accuracy_index"],[["df_all",df_all],["df_true",df_true],["df_false",df_all]],txt,valore_soglia)

In [ ]:
sns.boxplot(x="doc_unbiasedness_index", y="doc_accuracy_index", data=df_all)

In [ ]:
sns.boxplot(x="doc_precision_index", y="doc_accuracy_index", data=df_all)

In [ ]:
for df in [["df_all",df_all]]:
    print("-" * 100)
    print(df[0])
    df = pd.DataFrame(df[1].groupby("quartile").mean()[["doc_speaker_party_val","doc_precision_index","doc_unbiasedness_index","doc_accuracy_index","doc_speaker_trustworthiness_index","percentuale_parole_significative","numero_parole_frase","fre","sentiment", "subjectivity"]].reset_index(inplace=False))    
    df = df[(df["quartile"] != "other_true") & (df["quartile"] != "other_false") & (df["quartile"] != "other")].T
    df.columns = df.iloc[0].values.tolist()
    df = df.drop(df.index[0])
    
    
    display(df[["correct_false"	, "wrong_false","wrong_true","correct_true"	]])